In [4]:
# check if IS_MASTER exists, this variable will only exist if it's being called by MASTER notebook.
# if it does not exist, set it to False
try: IS_MASTER
except: IS_MASTER = False
# The code below will only run if it's NOT being called from MASTER notebook
if not IS_MASTER:
    # targetFiles = 'HealthyFood'
    # targetFiles = 'UnhealthyFood'
    targetFiles = 'smallData'
    DATA_DIR = './data/temp/' # 
    RAW_FILE = f'./data/CSVFiles/{targetFiles}-DataFrame.csv' # use the target file name here
    PROCESSED_FILE = f'{DATA_DIR}processed.pkl' # always use pickle for fast I/O!
    

generic = 'GenericTweets'

In [5]:
#Libraries used in the notebook
import csv
import json
import numpy as np
import pandas as pd
import glob
import os
import ntpath
import pickle
import shutil
import matplotlib.pyplot as plt
import sklearn


In [6]:
print("This is the preprocessing notebook being called from Master Notebook")


This is the preprocessing notebook being called from Master Notebook


In [15]:
chunksize = 10000

tfr = pd.read_csv(RAW_FILE, usecols=['id', 'text', 'full_text','text2Proc'], index_col=None, header=0, lineterminator='\n',chunksize=chunksize, iterator=True)
df_fastfood = pd.concat(tfr, ignore_index=True)
tfr = pd.read_csv(f'./data/CSVFiles/GenericTweets-Dataframe.csv', usecols=['id', 'text', 'full_text','text2Proc'], index_col=None, header=0, lineterminator='\n',chunksize=chunksize, iterator=True)
df_generic = pd.concat(tfr, ignore_index=True)

In [16]:
# df_healthy['target'] = 0
df_generic['target'] = 0
df_fastfood['target'] = 1
df_fastfood = df_fastfood.sample(n=500) #Modify number of samples depending on datasets size. We sample in order to have the 
df_generic = df_generic.sample(n=500)   # same number of samples of each class
df = pd.concat([df_generic, df_fastfood])
df.sample(n=10)

,id,text,full_text,text2Proc,target
1700,1.194622e+18,⚡‍ CHOLLAZO #Zapatos\n Zapatillas para Hombre...,NaN,⚡‍ CHOLLAZO #Zapatos\n Zapatillas para Hombre...,0
11362,1.194680e+18,Sucedió lo que todos esperábamos... Inició la ...,Sucedió lo que todos esperábamos... Inició la ...,Sucedió lo que todos esperábamos... Inició la ...,0
7160,1.199167e+18,@Mar_Masshiell @marianapcco Need galletas de a...,NaN,@Mar_Masshiell @marianapcco Need galletas de a...,1
13189,1.206024e+18,Y que no venga el rompepija de turno a joderme...,NaN,Y que no venga el rompepija de turno a joderme...,1
8133,1.194661e+18,"Información inútil sobre mi:\n\nAltura: 1, 64 ...","Información inútil sobre mi:\n\nAltura: 1, 64 ...","Información inútil sobre mi:\n\nAltura: 1, 64 ...",0
12765,1.194689e+18,Al no incluir esta pintura el tanque iba a red...,Al no incluir esta pintura el tanque iba a red...,Al no incluir esta pintura el tanque iba a red...,0
1093,1.193210e+18,bien sellado.\nahora en el horno con vino blan...,NaN,bien sellado.\nahora en el horno con vino blan...,1
4068,1.194639e+18,@luisbremer @Coto_Ar Luis hacete un lugar lo...,NaN,@luisbremer @Coto_Ar Luis hacete un lugar lo...,0
9607,1.194669e+18,"@pauroqa Veeeñ. Te esperamos con aguacate, gal...",NaN,"@pauroqa Veeeñ. Te esperamos con aguacate, gal...",0
6440,1.194653e+18,Daniela Martinez en la uva JAJAJAJAJA https://...,NaN,Daniela Martinez en la uva JAJAJAJAJA https://...,0


In [17]:
df.groupby('target').count()['text2Proc'] 

target
0    500
1    500
Name: text2Proc, dtype: int64

In [18]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 14286 to 3523
Data columns (total 5 columns):
id           1000 non-null float64
text         1000 non-null object
full_text    276 non-null object
text2Proc    1000 non-null object
target       1000 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 46.9+ KB


In [19]:
df.to_pickle(PROCESSED_FILE)